<a href="https://colab.research.google.com/github/vdslab/t-taiki0620/blob/main/Okapi_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
!pip install mojimoji

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

In [2]:
import os
import json
import csv
import pandas as pd
import MeCab
import subprocess
import mojimoji

path = '/content/drive/MyDrive/卒業演習/tsne_+_clusters_list.json'

with open(path, encoding='utf8')as f:
  df = pd.read_json(f)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22115 entries, 0 to 22114
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   22115 non-null  int64  
 1   プロジェクトID             22115 non-null  int64  
 2   公開年度                 22115 non-null  int64  
 3   事業番号1                22115 non-null  object 
 4   事業番号2                22115 non-null  int64  
 5   事業番号3                150 non-null    float64
 6   主要政策・施策              22115 non-null  object 
 7   府省庁                  22115 non-null  object 
 8   事業名                  22115 non-null  object 
 9   事業の目的                22115 non-null  object 
 10  事業概要                 22115 non-null  object 
 11  執行額                  22115 non-null  float64
 12  objective_x          22115 non-null  float64
 13  objective_y          22115 non-null  float64
 14  cluster_in_ministry  22115 non-null  int64  
 15  ALL                  22115 non-null 

In [3]:
df

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y,cluster_in_ministry,ALL,高齢社会対策,男女共同参画,少子化社会対策,子ども・若者育成支援,国土強靱化施策,障害者施策,地方創生,科学技術・イノベーション,ＩＴ戦略,一億総活躍推進,海洋政策,観光立国,宇宙開発利用,地球温暖化対策,ＯＤＡ,自殺対策,医療分野の研究開発関連,沖縄振興,知的財産,2020年東京オリパラ,犯罪被害者等施策,クールジャパン,食育推進,その他
0,13,16,2015,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、迅速・正確に内外に分かりやすく伝え...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,123.00,-3.427058,36.508163,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10604,16,2017,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,120.00,-3.443014,36.496967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,24982,16,2019,nan,21,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,106.00,-3.433842,36.497105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.00,-3.439199,36.503059,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,5377,16,2016,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、迅速・正確に内外に分かりやすく伝え...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,85.00,-3.429540,36.508717,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22110,22976,8983,2019,nan,117,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1628.00,-0.105321,-19.544006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22111,25938,44,2019,nan,150,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,30.73,28.377846,-26.652079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22112,25500,4895,2019,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,① 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7282.00,-7.667183,-13.079095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22113,13514,4916,2017,nan,38,NaN,['地方創生'],農林水産省,農林水産物の品目別輸出促進緊急対策事業,「総合的なＴＰＰ関連政策大綱」においては、高品質な我が国農林水産物の一層の輸出拡大により、「...,「農林水産業の輸出力強化戦略」及び「日本再興戦略2016」に基づき、コメ、青果物、花き、茶、...,431.00,-43.182682,-32.702126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.drop(columns=['高齢社会対策', '男女共同参画', '少子化社会対策', '子ども・若者育成支援', '国土強靱化施策', '障害者施策', '地方創生', '科学技術・イノベーション', 'ＩＴ戦略', '一億総活躍推進', '海洋政策', '観光立国', '宇宙開発利用', '地球温暖化対策', 'ＯＤＡ', '自殺対策', '医療分野の研究開発関連', '沖縄振興', '知的財産', '2020年東京オリパラ', '犯罪被害者等施策', 'クールジャパン', '食育推進', 'その他'])

In [5]:
df = df.drop(columns=['cluster_in_ministry', 'ALL'])

In [6]:
df_2018 = df.iloc[0:0]
print(df_2018)
cnt = 0
for i in range(len(df)):
  if df.loc[i]['公開年度'] == 2018:
    df_2018.loc[cnt] = df.loc[i]
    cnt += 1

Empty DataFrame
Columns: [ID, プロジェクトID, 公開年度, 事業番号1, 事業番号2, 事業番号3, 主要政策・施策, 府省庁, 事業名, 事業の目的, 事業概要, 執行額, objective_x, objective_y]
Index: []


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
df_2018

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y
0,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.0,-3.439199,36.503059
1,15755,15,2018,nan,1,NaN,['その他'],内閣官房,内閣人事局経費,政府全体を通じた成果向上及び人材育成を狙いとして実施する。\n各研修に応じ、①幹部候補育成課...,１．官民合同セミナー（幹部級）・(課長級)（幹部級：本府省の幹部職員並びに民間企業の取締役ク...,23.0,57.182091,-2.059250
2,15926,52,2018,nan,124,NaN,['その他'],内閣府,人道救援物資備蓄経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合等から人道的な国際救援活動への物資協力の要請をうけた場合、迅速に対応できるよう人道救...,59.0,46.140083,-57.230614
3,15964,35,2018,nan,162,NaN,['クールジャパン'],内閣府,クールジャパン戦略推進経費,ゲーム・マンガ・アニメといったコンテンツ、ファッション、産品、日本食、伝統文化、デザイン等、...,クールジャパン担当大臣等の海外出張を活用したクールジャパン発信イベント、クールジャパン戦略の...,49.0,2.651179,-55.060982
4,15924,84,2018,nan,122,NaN,['その他'],内閣府,国際平和協力隊の派遣等経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合の要請に基づき、引き続き南スーダンにおいて、国際平和協力業務を実施する。,78.0,46.272320,-57.395348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4368,20146,3574,2018,nan,32,NaN,['地球温暖化対策'],環境省,短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金関連業務,国内におけるブラックカーボン等の短期寿命気候汚染物質（SLCP）の排出実態及び対策技術等に係...,１．短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金\n CCACに対し、平成24...,327.0,-10.464338,-70.462219
4369,15941,44,2018,nan,139,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,21.6,28.345156,-26.662083
4370,18840,4895,2018,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,１ 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7886.0,-7.659595,-13.079552
4371,19154,3708,2018,nan,119,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1454.0,-0.068404,-19.573669


In [8]:
import re
def normalize_text(text):
  # 全部全角に変換
  result = mojimoji.han_to_zen(text)

  # 全角から半角に変換（カナは除く）
  result = mojimoji.zen_to_han(result, kana=False)

  # 全ての文字を小文字に変換
  result = result.lower()
  
  return result
  # return re.sub(' ', '', mojimoji.zen_to_han(text)).upper().split()

In [9]:
# 0404 used this
cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
def tokenize(text):
  m=MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
  lines = m.parse(text).splitlines()
  words = []
  for line in lines:
    chunks = line.split('\t')
    # print(chunks[0])
    if chunks[0] != 'EOS' and not chunks[1].startswith('記号'):
      words.append(chunks[0])
  return words

In [10]:
from gensim.models import word2vec
model   = word2vec.Word2Vec.load("/content/drive/MyDrive/Colab Notebooks/pre-judgit/ja-MeCab-50.data.model")

In [11]:
#分かち書きする文
separation = "事業の目的"
#separation = "事業概要"

In [12]:
tf_business_list = []
word_list = []
for i in range(len(df_2018)):
  tf_business_list.append(df_2018.loc[i])
  if df_2018.loc[i][separation] != '':
    if normalize_text(df_2018.loc[i][separation]) != []:
      words = tokenize(normalize_text(df_2018.loc[i][separation]))
      for word in words:
        if word not in word_list:
          word_list.append(word)

In [13]:
print(len(tf_business_list), len(word_list))
print(tf_business_list[0])
print(word_list)

4373 13771
ID                                                         15765
プロジェクトID                                                      16
公開年度                                                        2018
事業番号1                                                        nan
事業番号2                                                         11
事業番号3                                                        NaN
主要政策・施策                                                  ['その他']
府省庁                                                         内閣官房
事業名                                                       内閣広報経費
事業の目的          内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...
事業概要           内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...
執行額                                                           77
objective_x                                              -3.4392
objective_y                                              36.5031
Name: 0, dtype: object
['内閣', 'の', '重要', '施策', 'に関する', '広報', 'を', '首相官邸', 'ホームペ

In [14]:
words = tokenize(normalize_text(tf_business_list[0].loc[separation])[0])

In [15]:
tf = {}
for data in tf_business_list:
  tf[data.loc["プロジェクトID"]] = {}
  words = tokenize(normalize_text(data.loc[separation]))
  tmp = {}
  for word in words:
    tmp[word] = words.count(word)/len(words)
  tf[data.loc["プロジェクトID"]] = tmp

In [ ]:
tf[4924]

{'ある': 0.012658227848101266,
 'い': 0.012658227848101266,
 'いく': 0.012658227848101266,
 'から': 0.012658227848101266,
 'が': 0.0379746835443038,
 'こと': 0.02531645569620253,
 'し': 0.02531645569620253,
 'する': 0.05063291139240506,
 'た': 0.02531645569620253,
 'ため': 0.02531645569620253,
 'て': 0.02531645569620253,
 'で': 0.02531645569620253,
 'ない': 0.012658227848101266,
 'に': 0.0379746835443038,
 'において': 0.012658227848101266,
 'により': 0.012658227848101266,
 'の': 0.0379746835443038,
 'は': 0.012658227848101266,
 'や': 0.02531645569620253,
 'れ': 0.012658227848101266,
 'を': 0.06329113924050633,
 'システム': 0.012658227848101266,
 'モデル': 0.012658227848101266,
 '不足': 0.012658227848101266,
 '中': 0.012658227848101266,
 '作業量': 0.012658227848101266,
 '力': 0.012658227848101266,
 '労働': 0.012658227848101266,
 '労働力': 0.02531645569620253,
 '化': 0.012658227848101266,
 '各': 0.012658227848101266,
 '大規模化': 0.012658227848101266,
 '就業': 0.012658227848101266,
 '平準': 0.012658227848101266,
 '年間': 0.012658227848101266,
 '必要': 

In [16]:
df_2018

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y
0,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.0,-3.439199,36.503059
1,15755,15,2018,nan,1,NaN,['その他'],内閣官房,内閣人事局経費,政府全体を通じた成果向上及び人材育成を狙いとして実施する。\n各研修に応じ、①幹部候補育成課...,１．官民合同セミナー（幹部級）・(課長級)（幹部級：本府省の幹部職員並びに民間企業の取締役ク...,23.0,57.182091,-2.059250
2,15926,52,2018,nan,124,NaN,['その他'],内閣府,人道救援物資備蓄経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合等から人道的な国際救援活動への物資協力の要請をうけた場合、迅速に対応できるよう人道救...,59.0,46.140083,-57.230614
3,15964,35,2018,nan,162,NaN,['クールジャパン'],内閣府,クールジャパン戦略推進経費,ゲーム・マンガ・アニメといったコンテンツ、ファッション、産品、日本食、伝統文化、デザイン等、...,クールジャパン担当大臣等の海外出張を活用したクールジャパン発信イベント、クールジャパン戦略の...,49.0,2.651179,-55.060982
4,15924,84,2018,nan,122,NaN,['その他'],内閣府,国際平和協力隊の派遣等経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合の要請に基づき、引き続き南スーダンにおいて、国際平和協力業務を実施する。,78.0,46.272320,-57.395348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4368,20146,3574,2018,nan,32,NaN,['地球温暖化対策'],環境省,短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金関連業務,国内におけるブラックカーボン等の短期寿命気候汚染物質（SLCP）の排出実態及び対策技術等に係...,１．短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金\n CCACに対し、平成24...,327.0,-10.464338,-70.462219
4369,15941,44,2018,nan,139,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,21.6,28.345156,-26.662083
4370,18840,4895,2018,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,１ 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7886.0,-7.659595,-13.079552
4371,19154,3708,2018,nan,119,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1454.0,-0.068404,-19.573669


In [ ]:
len(word_list)

13771

In [17]:
import math
tmp = {}
for data in tf_business_list:
  words = tokenize(normalize_text(data.loc[separation]))
  check = []
  for word in words:
    if word not in check:
      check.append(word)
      if word not in tmp.keys():
        tmp[word] = 1
      else:
        tmp[word] += 1

In [ ]:
tmp

In [18]:
idf = {}
for word in word_list:
  idf[word] = math.log2((len(tf_business_list)-tmp[word]+0.5)/(tmp[word]+0.5))

In [ ]:
idf

In [19]:
sum = 0
for word in word_list:
  sum += tmp[word]

In [ ]:
sum

200800

In [20]:
avgdl = sum/len(tf_business_list)

In [ ]:
avgdl

45.91813400411617

In [21]:
okapi = {}
count = 0
for data in tf_business_list:
  if data.loc['プロジェクトID'] not in okapi.keys():
    okapi[data.loc['プロジェクトID']] = {}
  words = tokenize(normalize_text(data.loc[separation]))
  ex = {}
  # print(count, data.loc['プロジェクトID'], data.loc['事業名'], words)
  count += 1
  for word in words:
    ex[word] = idf[word] * ((tf[data.loc['プロジェクトID']][word]*(2.0+1)) / (tf[data.loc['プロジェクトID']][word] + 2.0 * (1-0.75+0.75*(tmp[word]/avgdl))))
  
  # print(ex)
  okapi[data.loc['プロジェクトID']] = ex

In [22]:
result_sorted = sorted(okapi[15].items(), key=lambda x: x[1], reverse=True)

In [ ]:
result_sorted

In [23]:
word_score = []
tmp_list = sorted(okapi[15].items(), key=lambda x: x[1], reverse=True)
for i in range(5):
    word_score.append({"word" : tmp_list[i][0],
                     "score" : tmp_list[i][1]})
print(word_score)

[{'word': '狙い', 'score': 0.8825632320938026}, {'word': '課程', 'score': 0.587474312036277}, {'word': '国家公務員', 'score': 0.5199907987137709}, {'word': '幹部', 'score': 0.4661409328412976}, {'word': '候補', 'score': 0.4661409328412976}]


In [ ]:
import json
obj = []
for data in tf_business_list:
  print(data.loc['プロジェクトID'])
  tmp_array = []
  tmp_list = sorted(okapi[data.loc['プロジェクトID']].items(), key=lambda x : x[1], reverse=True)
  for i in range(5):
    try:
      tmp_array.append({"word": tmp_list[i][0],
                         "score":float(tmp_list[i][1])})
    except:
      print("skip")
  obj.append({'プロジェクトID':int(data.loc['プロジェクトID']),
              'okapi':tmp_array})

with open('/content/drive/MyDrive/Colab Notebooks/pre-judgit/okapi2018.json', 'w', encoding='utf8')as f:
  json.dump(obj, f, indent=2, ensure_ascii=False)

In [25]:
okapi[3550]

{'における': 0.03943625453091387,
 '中央合同庁舎第5号館': 11.028907532246103,
 '工事': 3.5491869276043877,
 '等': -0.007710373336530736}

In [ ]:
len(obj)

4373

In [ ]:
for data in tf_business_list:
  if data.loc["プロジェクトID"] == 469:
    print(data)

ID                                                         18069
プロジェクトID                                                     469
公開年度                                                        2018
事業番号1                                                        nan
事業番号2                                                        436
事業番号3                                                        NaN
主要政策・施策                                                  ['その他']
府省庁                                                        厚生労働省
事業名                                                 労災保険給付に必要な経費
事業の目的          　労災保険給付は、労働者の業務上又は通勤による負傷、疾病、障害、死亡等に対して、迅速かつ公正...
事業概要                                                    　別添のとおり。
執行額                                                       737992
objective_x                                              25.1803
objective_y                                              81.7496
Name: 722, dtype: object
